# Reference

### 코드
https://github.com/donghwan2/Text_analysis/blob/master/%EB%84%A4%EC%9D%B4%EB%B2%84%20%EC%98%81%ED%99%94%20%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D%20%EB%B0%8F%20%EA%B8%8D%EC%A0%95%2C%EB%B6%80%EC%A0%95%20%EC%98%88%EC%B8%A1%20%EB%AA%A8%EB%8D%B8.ipynb

https://github.com/e9t/nsmc/


### TfidfVectorizer
https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/

https://thinkwarelab.wordpress.com/2016/11/14/ir-tf-idf-%EC%97%90-%EB%8C%80%ED%95%B4-%EC%95%8C%EC%95%84%EB%B4%85%EC%8B%9C%EB%8B%A4/

https://blog.breezymind.com/2018/03/02/sklearn-feature_extraction-text-2/

https://m.blog.naver.com/PostView.nhn?blogId=samsjang&logNo=220985170721&proxyReferer=https%3A%2F%2Fwww.google.com%2F

In [4]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import os
from urllib import parse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
from konlpy.tag import *
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
def step1_get_detail_url() :
    site = 'https://movie.naver.com/movie/running/current.nhn?order=reserve'
    # 현재 상영작 홈페이지 url
    response = requests.get(site)   
    # 사이트에 접속합니다.
    bs = BeautifulSoup(response.content, 'html.parser')
    # html 코드를 가져옵니다
    a_list = bs.select('.top_thumb_lst a')
    df = pd.DataFrame()
    
    for idx in range(10) :       # 상위 10개만 가져오기
        href = a_list[idx].get('href')
        a1 = parse.urlparse(href)
        query_str = parse.parse_qs(a1.query)
        code = query_str['code'][0]
        df = df.append([[code]], ignore_index=True)
        # html 코드에서 영화 코드를 가져옵니다.

    df.columns = ['code']
    df.to_csv('movie_code_list.csv', index=False, encoding='utf-8-sig')
    # movie_code_list에 영화 코드를 저장합니다.

# 영화 정보페이지로 이동하기 위해서 필요한 함수입니다

In [3]:
def step2_get_reple_href() :
    code_frame = pd.read_csv('movie_code_list.csv')
    # 영화 코드 파일을 가져옵니다.
    code_list = code_frame['code'].tolist()
    url_list = pd.DataFrame()
    # url 주소를 담을 dataframe을 만듭니다.
    
    for code in code_list:
        site = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=%s&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false' % code
        # 파이썬 문법의 특징인데 https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=%s
        # 이부분에 %s 로 된 부분을 뒤에 % code 값으로 대체합니다.(포맷팅이라고 합니다.)
        # code는 영화 코드이므로 영화 코드에 맞게 영화 상세 페이지로 이동합니다.
        url_list = url_list.append([[site]], ignore_index=True)

    url_list.columns = ['url']
    url_list.to_csv('movie_url_list.csv', index=False, encoding='utf-8-sig')

# 영화 정보 페이지 url 주소들을 movie_url_list로 저장합니다

In [4]:
def step3_get_reply_data() :
    df = pd.read_csv('movie_url_list.csv')
    # 영화 정보 페이지 url 주소가 담겨있는 파일을 가져옵니다.
    url_list = df['url'].tolist()
    
    for url in url_list :
        response = requests.get(url)
        # 사이트에 접속합니다.
        bs = BeautifulSoup(response.content, 'html.parser')
        # html 코드를 가져옵니다
        
        total = bs.find('div',class_='score_total').findChildren('em')[0].getText()
        # 전체 리뷰 개수를 가져옵니다.
        score_total = int(total.replace(',', ''))    # 쉼표 없애기 / int(정수형)로 만들기
        pageCnt = score_total // 10     # 한페이지당 10개의 리뷰가 있어서
        if score_total % 10 > 0 :
            pageCnt += 1

        # 전체 페이지를 돌면서 리뷰를 가져온다.
        # 현재 페이지
        now_page = 1

        pageCnt = 100
        # 영화 하나당 리뷰 100개페이지까지 수집. (1000개)
        while now_page <= pageCnt :
            sleep(1)
            url2 = url + '&page=' + str(now_page)
            # 요청할 페이지의 주소
            
            response2 = requests.get(url2)
            bs2 = BeautifulSoup(response2.content, 'html.parser')

            result_df = pd.DataFrame()
            # 리뷰 데이터를 추출합니다.
            
            lis = bs2.select('.score_result li')
            # li 태그들을 가져옵니다.(score_reple 태그-리뷰-를 포함하고 있는)
            for obj in lis :
            # score_result에는 한페이지에 10개의 리뷰가 있기떄문에 10개의 li가 존재합니다
                star_score = obj.select('.star_score em')[0].text
                # 평점
                score_reple = obj.select('.score_reple p')[0].text
                # 리뷰
                score_reple = score_reple.replace('\t','').replace('\r','').replace('\n','')
                # 필요없는 부분을 제거합니다.
                
                result_df = result_df.append([[score_reple, star_score]], ignore_index=True)

            if os.path.exists('star_score.csv') == False :     # 아직 파일이 없으면 파일을 만듭니다.
                result_df.columns = ['text', 'score']
                result_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig')
            else :                                               # 이미 파일이 있으면 결과를 더해줍니다.
                result_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

            now_page += 1
            # 한 페이지가 끝나면 페이지수를 하나씩 늘려줍니다

In [5]:
# 스크래핑 함수
def scrapping() :

    # 각 영화 코드 데이터를 가져와 저장합니다.
    step1_get_detail_url()

    # 리뷰 데이터가 있는 페이지의 주소를 저장합니다.
    step2_get_reple_href()

    # 리뷰 데이터를 가져옵니다
    step3_get_reply_data()

In [6]:
scrapping()

In [7]:
test_df = pd.read_csv('star_score.csv')

In [8]:
test_df.head(10)

,text,score
0,관람객보고나면 르네가 왜 아카데미 여주상인지 바로 알 수 있다는 ㅜㅜ 연기 쩔고 오...,10
1,영화 끝나고 눈물이 눈물이 ㅜㅜ 정말 슬픈 삶을을 살았네요 주디 갈란드~먹고 싶은거...,10
2,모두의 사랑을 받았으나 자신을 사랑하는 법을 몰랐던 '어른아이'의 아련한 비가(悲歌).,8
3,관람객생전 모르던 사람이 이렇게 안타까울수가...,10
4,관람객반짝이는 주디 글자가 너무 슬프네요. 영화 끝나고도 여운이 안가셔서 밤새 같이...,10
5,오즈의 마법사의 어린 소녀 도로시의 비극적인 삶이 잘드러나는 영화였다. 내가 어릴적...,10
6,관람객오버더레인보우 나올때 눈물이.. 르네 젤위거 짱,10
7,르네 젤위거 명연기..,10
8,이런 영화 개봉 좀 해주지.,10
9,르네젤위거를 브리짓존스로만 기억했었던걸 반상한다ㅠㅠ연기폭이 이렇게 넓은 배우였구나 ...,10


In [5]:
# 리뷰 데이터 전처리 함수
def text_preprocessing(text) :
    if text.startswith('관람객') :
        return text[3:]
    # 일부 관람객 으로 시작하는 리뷰가 있어서 관람객이라는 부분을 제거해줍니다.
    # 실제 리뷰 페이지를 가보시면 확인할 수 있습니다.
    # https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=189001&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
    
    elif text.startswith('스포일러가 포함된 감상평입니다.'):
        return text[24:]
    # 마찬가지로 스포일라거 포함된 감상평입니다.감상평 보기 라고 시작하는 부분이있습니다.
    # 제거해줍니다
    else :
        return text
    
# 평점 전처리 함수
def star_preprocessing(text) :
    value = int(text)

    if value > 8:
        return '2' # 9,10 긍정
    elif value < 5:
        return '0' # 1,2,3,4 부정
    else:
        return '1'# 5,6,7,8 중립

In [6]:
def step4_data_preprocessing() :
    df = pd.read_csv('star_score.csv')
    
    df['text'] = df['text'].astype(str)
    # text데이터가 float형태로 저장되어있어 str형태로 변환해주니다.
    df['text'].apply(text_preprocessing)
    # 리뷰데이터 전처리를 실행합니다.
    df['score'] = df['score'].apply(star_preprocessing)
    # 평점데이터 전처리를 실행합니다.
    
    text_list = df['text'].tolist()
    star_list = df['score'].tolist()

    text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.3, random_state=0)
    # 리뷰 데이터와 평점 데이터를 각각 7:3으로 나누고 70%는 학습 데이터 30% 테스트 데이터로 사용합니다.

    return text_train, text_test, star_train, star_test
    # split한 데이터를 반환합니다.

In [7]:
def tokenizer(text) :
    okt = Okt()
    return okt.morphs(text)
# okt라이브러리를 사용하여 형태소 분석을 실시합니다.

In [8]:
def step5_learning(X_train, y_train, X_test, y_test):
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
    # TfidfVectorizer 방식으로 토큰화된 단어들의 tf-idf값을 계산합니다.
    # lowercase : 소문자로 바꾸는 파라미터(한글 텍스트라서 필요없기떄문에 false)

    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)
    # logisticregression 

    pipe = Pipeline([('vect', tfidf), ('clf', logistic)])
    # vecter 데이터는 tfidf, 분류 데이터는 logisticgression으로 학습합니다

    pipe.fit(X_train, y_train)
    # train 데이터를 사용하여 학습합니다.

    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    # 모델의 정확도를 측정합니다

    with open('pipe.dat', 'wb') as fp :
        pickled_data = pickle.dump(pipe, fp)
    # 모델을 추후에 사용하기위해 pickle 이라는 데이터 저장 라이브러리를 사용하여 저장합니다.

In [9]:
def learning() :
    text_train, text_test, star_train, star_test = step4_data_preprocessing()
    # 리뷰 데이터의 70%는 text_train 30% text_test
    # 평점 데이터의 70% star_train 30%는 star_test로 전처리를 시행합니다.
    step5_learning(text_train, star_train, text_test, star_test)
    # 학습을 진행합니다.

In [10]:
learning()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8086714399363564


In [15]:
def step6_using_model() :
    
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)
    # 이전에 만들어 놓은 모델을 다시 블러옵니다.
    
    while True :
        text = input('리뷰를 작성해주세요 :')
        # 문자를 입력할 수 있는 창을 만듭니다.

        str = [text]

        r1 = np.max(pipe.predict_proba(str) * 100)
        # 예측 정확도
        r2 = pipe.predict(str)[0]
        # 예측 결과
        
        if text =='그만':
            break
        
        if r2 == '2' :
            print('긍정적인 리뷰')
        elif r2 =="0" :
            print('부정적인 리뷰')
        elif r2 =='1' :
            print('중립적인 리뷰')
        print('정확도 : %.3f' % r1)
        


In [16]:
def using() :
    step6_using_model()

In [17]:
using()

리뷰를 작성해주세요 :좋아요
긍정적인 리뷰
정확도 : 88.454
리뷰를 작성해주세요 :재미없어요
부정적인 리뷰
정확도 : 95.438
리뷰를 작성해주세요 :노잼
부정적인 리뷰
정확도 : 99.083
리뷰를 작성해주세요 :그저그래요
중립적인 리뷰
정확도 : 63.698
리뷰를 작성해주세요 :그닥
부정적인 리뷰
정확도 : 88.102
리뷰를 작성해주세요 :그만
